# Merged notebooks for Not Supported CIViC and MOA evidence (assertion analysis)

In [ ]:
from pathlib import Path
from enum import Enum
import zipfile
import pandas as pd
from civicpy import civic as civicpy
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.pyplot import figure
from typing import Dict
import json
import requests
from ga4gh.core import sha512t24u #pip install ga4gh.vrs
import itertools

## Merge CIViC and MOA Summary Variant Dataframes

In [ ]:
civic_variant_ids = civicpy.get_all_variants()
total_number_variants = len(civic_variant_ids)
f"Total Number of variants in CIViC: {total_number_variants}"

In [ ]:
def get_feature_digest(feature: Dict) -> str:
    """Get digest for feature

    :param feature: MOA feature
    :return: Digest
    """
    attrs = json.dumps(
        feature["attributes"][0],
        sort_keys=True,
        separators=(",", ":"),
        indent=None
    ).encode("utf-8")
    return sha512t24u(attrs)

In [ ]:
# Create dictionary for MOA Feature ID -> Feature Type
r = requests.get("https://moalmanac.org/api/features")
if r.status_code == 200:
    feature_data = r.json()

features = {}

for feature in feature_data:
    feature_id = feature["feature_id"]
    digest = get_feature_digest(feature)
    features[digest] = feature["feature_type"]

count_unique_feature_ids = len(features.keys())
f"Total Number of variants in MOA: {count_unique_feature_ids}"

In [ ]:
#add all variants from CIViC and MOA
merged_variant_total = total_number_variants+count_unique_feature_ids
merged_variant_total =float(merged_variant_total)
merged_variant_total

In [ ]:
#import summary tables from source notebooks
for_merge_all_variant_percent_of_civic_df = pd.read_csv("../analysis/for_merge_all_variant_percent_of_civic_df.csv", sep=",")
for_merge_all_variant_percent_of_moa_df = pd.read_csv("../analysis/for_merge_all_variant_percent_of_moa_df.csv", sep=",")

In [ ]:
#merge CIViC and MOA summary tables
merged_all_variants_df = pd.merge(for_merge_all_variant_percent_of_civic_df, for_merge_all_variant_percent_of_moa_df, on="Variant Category",how="outer")
merged_all_variants_df = merged_all_variants_df.replace(np.nan, 0, regex=True)

Summary Table 1: The table below shows the 3 categories that CIViC and MOA variants were divided into after normalization and what percentage they make up of all variants in the respective source data. 

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_1 = merged_all_variants_df.drop(["Count of CIViC Variant Items per Category", "Count of MOA Features per Category"], axis=1)
merged_civic_moa_summary_table_1 = merged_civic_moa_summary_table_1.set_index("Variant Category")
merged_civic_moa_summary_table_1

In [ ]:
#add up variants from CIViC and MOA for each Variant Category
merged_all_variants_df["Count of CIViC Variant Items per Category"] = merged_all_variants_df["Count of CIViC Variant Items per Category"].astype(float)
merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"] = merged_all_variants_df["Count of CIViC Variant Items per Category"]+ merged_all_variants_df["Count of MOA Features per Category"]

In [ ]:
#new column for the total combined variant number from CIViC and MOA
merged_all_variants_df["Sum of total Variants from CIViC and MOA"]= merged_variant_total

In [ ]:
#new percentage of each category of the total merged variants from CIViC and MOA
merged_all_variants_df["Merged Variant Percentage"]= (merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"] / merged_all_variants_df["Sum of total Variants from CIViC and MOA"])*100
merged_all_variants_df=merged_all_variants_df.round({"Merged Variant Percentage":2})
merged_all_variants_df["Merged Variant Percentage"]= merged_all_variants_df["Merged Variant Percentage"].astype(str) + "%"

In [ ]:
#convert to integers to remove decimals
merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"] = merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"].astype(int)

merged_all_variants_df["Sum of total Variants from CIViC and MOA"] = merged_all_variants_df["Sum of total Variants from CIViC and MOA"].astype(int)

In [ ]:
#merge fraction and percentage
merged_all_variants_df["Percentage of all Merged Variant Items"]= merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"].astype(str)+" / "+merged_all_variants_df["Sum of total Variants from CIViC and MOA"].astype(str)+" ("+merged_all_variants_df["Merged Variant Percentage"].astype(str)+")"

Summary Table 2: The table below shows the 3 categories that merged CIViC and MOA variants were divided into after normalization and what percentage they make up of all variants in the combined data. 

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_2 = merged_all_variants_df.drop(["Count of CIViC Variant Items per Category", "Percentage of all CIViC Variant Items", "Count of MOA Features per Category", "Percentage of all MOA Features", "Sum of Variants from CIViC and MOA per Category", "Sum of total Variants from CIViC and MOA","Merged Variant Percentage" ], axis=1)
merged_civic_moa_summary_table_2= merged_civic_moa_summary_table_2.set_index("Variant Category")
merged_civic_moa_summary_table_2

In [ ]:
#convert to integers to remove decimals
merged_all_variants_df["Count of CIViC Variant Items per Category"]= merged_all_variants_df["Count of CIViC Variant Items per Category"].astype(int)

merged_all_variants_df["Count of MOA Features per Category"]= merged_all_variants_df["Count of MOA Features per Category"].astype(int)

In [ ]:
#new percentage of CIViC contribution for each category
merged_all_variants_df["CIViC Variants of Category Percentage"]=(merged_all_variants_df["Count of CIViC Variant Items per Category"] / merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"])*100
merged_all_variants_df=merged_all_variants_df.round({"CIViC Variants of Category Percentage":2})
merged_all_variants_df["CIViC Variants of Category Percentage"]= merged_all_variants_df["CIViC Variants of Category Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_all_variants_df["Percentage of CIViC Variants of Category"]=merged_all_variants_df["Count of CIViC Variant Items per Category"].astype(str)+" / "+merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"].astype(str)+" ("+merged_all_variants_df["CIViC Variants of Category Percentage"].astype(str)+")"

In [ ]:
#new percentage of MOA contribution for each category
merged_all_variants_df["MOA Variants of Category Percentage"]=(merged_all_variants_df["Count of MOA Features per Category"] / merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"])*100
merged_all_variants_df=merged_all_variants_df.round({"MOA Variants of Category Percentage":2})
merged_all_variants_df["MOA Variants of Category Percentage"]= merged_all_variants_df["MOA Variants of Category Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_all_variants_df["Percentage of MOA Variants of Category"]=merged_all_variants_df["Count of MOA Features per Category"].astype(str)+" / "+merged_all_variants_df["Sum of Variants from CIViC and MOA per Category"].astype(str)+" ("+merged_all_variants_df["MOA Variants of Category Percentage"].astype(str)+")"

Summary Table 3: The table below shows what percentage of the merged data originates from CIViC and MOA sources

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_3 = merged_all_variants_df.drop(["Count of CIViC Variant Items per Category", "Percentage of all CIViC Variant Items", "Count of MOA Features per Category", "Percentage of all MOA Features", "Sum of Variants from CIViC and MOA per Category", "Sum of total Variants from CIViC and MOA","Merged Variant Percentage","Percentage of all Merged Variant Items","CIViC Variants of Category Percentage", "MOA Variants of Category Percentage" ], axis=1)
merged_civic_moa_summary_table_3= merged_civic_moa_summary_table_3.set_index("Variant Category")
merged_civic_moa_summary_table_3

In [ ]:
#import summary tables from source notebooks
for_merge_not_supported_variant_percent_of_civic_df = pd.read_csv("../analysis/for_merge_not_supported_variant_percent_of_civic_df.csv", sep=",")
for_merge_not_supported_features_total_df = pd.read_csv("../analysis/for_merge_not_supported_features_total_df.csv", sep=",")

In [ ]:
#merge CIViC and MOA summary tables
merged_not_supported_variants_df = pd.merge(for_merge_not_supported_variant_percent_of_civic_df, for_merge_not_supported_features_total_df, on="Category",how="outer")
merged_not_supported_variants_df = merged_not_supported_variants_df.replace(np.nan, 0, regex=True)

Summary Table 4: The table below shows the categories of Not Supported variants and what percentage of source (CIViC or MOA) variants they make up. These percentages will not add up to 100% because Not Supported variants make up 45.40% of CIViC variants and 57.01% of MOA variants.(Reference Summary Table 1- merged_civic_moa_summary_table_1)

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_3= merged_not_supported_variants_df.drop(["Count of CIViC Variant Items per Category","Count of MOA Features per Category"], axis=1)
merged_civic_moa_summary_table_3= merged_civic_moa_summary_table_3.set_index("Category")
merged_civic_moa_summary_table_3

In [ ]:
#add up evidence from CIViC and MOA for each Variant Category
merged_not_supported_variants_df["Sum of Variants from CIViC and MOA per Category"]=merged_not_supported_variants_df["Count of CIViC Variant Items per Category"]+merged_not_supported_variants_df["Count of MOA Features per Category"]
merged_not_supported_variants_df["Sum of total Variants from CIViC and MOA"]= int(merged_variant_total)

In [ ]:
#new percentage of each category of the total merged variants from CIViC and MOA
merged_not_supported_variants_df["Merged Not Supported Variant Category of Merged Total Percentage"]= (merged_not_supported_variants_df["Sum of Variants from CIViC and MOA per Category"] / merged_not_supported_variants_df["Sum of total Variants from CIViC and MOA"])*100
merged_not_supported_variants_df=merged_not_supported_variants_df.round({"Merged Not Supported Variant Category of Merged Total Percentage":2})
merged_not_supported_variants_df["Merged Not Supported Variant Category of Merged Total Percentage"]= merged_not_supported_variants_df["Merged Not Supported Variant Category of Merged Total Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_not_supported_variants_df["Percentage of all Merged Variant Items"]= merged_not_supported_variants_df["Sum of Variants from CIViC and MOA per Category"].astype(str)+" / "+merged_not_supported_variants_df["Sum of total Variants from CIViC and MOA"].astype(str)+" ("+merged_not_supported_variants_df["Merged Not Supported Variant Category of Merged Total Percentage"].astype(str)+")"

Summary Table 5: The table below shows the categories of Not Supported variants and what percentage of all variants (CIViC and MOA) they make up. These percentages will not add up to 100% because Not Supported variants make up 46.64% of all variants (CIViC and MOA). 

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_5 = merged_not_supported_variants_df.drop(["Count of CIViC Variant Items per Category", "Percent of all CIViC Variant Items", "Count of MOA Features per Category", "Percent of all MOA Features", "Sum of Variants from CIViC and MOA per Category", "Sum of total Variants from CIViC and MOA", "Merged Not Supported Variant Category of Merged Total Percentage"], axis=1)
merged_civic_moa_summary_table_5=merged_civic_moa_summary_table_5.set_index("Category")
merged_civic_moa_summary_table_5

In [ ]:
#add up all variants, for total number of Not Supported Variants
merged_not_supported_variant_total= merged_not_supported_variants_df["Sum of Variants from CIViC and MOA per Category"].sum()
merged_not_supported_variants_df["Merged Not Supported Variant Total"]=merged_not_supported_variant_total

In [ ]:
#new percentage of each category of the total merged variants from CIViC and MOA
merged_not_supported_variants_df["Merged Not Supported Variant Category of Not Supported Percentage"]= (merged_not_supported_variants_df["Sum of Variants from CIViC and MOA per Category"] / merged_not_supported_variants_df["Merged Not Supported Variant Total"])*100
merged_not_supported_variants_df=merged_not_supported_variants_df.round({"Merged Not Supported Variant Category of Not Supported Percentage":2})
merged_not_supported_variants_df["Merged Not Supported Variant Category of Not Supported Percentage"]= merged_not_supported_variants_df["Merged Not Supported Variant Category of Not Supported Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_not_supported_variants_df["Percentage of all Not Supported Variant Items"]= merged_not_supported_variants_df["Sum of Variants from CIViC and MOA per Category"].astype(str)+" / "+merged_not_supported_variants_df["Merged Not Supported Variant Total"].astype(str)+" ("+merged_not_supported_variants_df["Merged Not Supported Variant Category of Not Supported Percentage"].astype(str)+")"

Summary Table 6: The table below shows the categories of Not Supported variants the percent of each category of total merged Not Supported variants.

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_6 = merged_not_supported_variants_df.drop(["Count of CIViC Variant Items per Category", "Percent of all CIViC Variant Items", "Count of MOA Features per Category", "Percent of all MOA Features", "Sum of Variants from CIViC and MOA per Category", "Sum of total Variants from CIViC and MOA","Percentage of all Merged Variant Items","Merged Not Supported Variant Category of Merged Total Percentage", "Merged Not Supported Variant Total","Merged Not Supported Variant Category of Not Supported Percentage"], axis=1)
merged_civic_moa_summary_table_6=merged_civic_moa_summary_table_6.set_index("Category")
merged_civic_moa_summary_table_6

## Merge CIViC and MOA Summary Evidence Dataframes

In [ ]:
civic_evidence_ids = civicpy.get_all_evidence(include_status=["accepted", "submitted"])

In [ ]:
total_ac_sub_evidence = len(civic_evidence_ids)
f"Total Number of accepted and submitted evidence items in CIViC: {total_ac_sub_evidence}"

In [ ]:
# Create DF for assertions and their associated feature + predictive implication
r = requests.get("https://moalmanac.org/api/assertions")
if r.status_code == 200:
    assertion_data = r.json()

transformed = []

# Mapping from feature ID to feature digest
feature_id_to_digest = {}

for assertion in assertion_data:
    assertion_id = assertion["assertion_id"]
    predictive_implication = assertion["predictive_implication"]

    if len(assertion["features"]) != 1:
        print(f"assertion id ({assertion_id}) does not have 1 feature")
        continue

    feature = assertion["features"][0]
    feature_id = feature["feature_id"]
    feature_digest = get_feature_digest(feature)

    feature_id_to_digest[feature_id] = digest

    transformed.append(
        {
            "assertion_id": assertion_id,
            "feature_id": feature_id,
            "feature_type": features[feature_digest],
            "predictive_implication": predictive_implication,
            "feature_digest": feature_digest
        }
    )
moa_df = pd.DataFrame(transformed)

In [ ]:
total_len_assertions = len(moa_df.assertion_id.unique())
f"Total number of unique assertions (evidence items): {total_len_assertions}"

In [ ]:
#add all evidence items(assertions) from CIViC and MOA
merged_evidence_total = total_ac_sub_evidence +total_len_assertions

In [ ]:
#import summary tables from source notebooks
for_merge_all_variant_evidence_percent_of_civic_df = pd.read_csv("../analysis/for_merge_all_variant_evidence_percent_of_civic_df.csv", sep=",")
for_merge_all_features_assertions_df = pd.read_csv("../analysis/for_merge_all_features_assertions_df.csv", sep=",")

In [ ]:
#merge CIViC and MOA summary tables
merged_all_evidence_df = pd.merge(for_merge_all_variant_evidence_percent_of_civic_df, for_merge_all_features_assertions_df, on="Variant Category",how="outer")
merged_all_evidence_df = merged_all_evidence_df.replace(np.nan, 0, regex=True)
merged_all_evidence_df["Count of MOA Assertions per Category"]=merged_all_evidence_df["Count of MOA Assertions per Category"].astype(int)


Summary Table 7: The table below shows what percentage of all evidence items in CIViC and MOA are associated with Normalized, Unable to Normalize, and Not Supported variants. This will not add up to 100% because evidence itmes may be used across multiple variants.

In [ ]:
merged_civic_moa_summary_table_7 = merged_all_evidence_df.drop(["Count of CIViC Evidence Items per Category", "Count of MOA Assertions per Category"], axis=1)
merged_civic_moa_summary_table_7 = merged_civic_moa_summary_table_7.set_index("Variant Category")
merged_civic_moa_summary_table_7

In [ ]:
#add up variants from CIViC and MOA for each Variant Category
merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"] = merged_all_evidence_df["Count of CIViC Evidence Items per Category"]+ merged_all_evidence_df["Count of MOA Assertions per Category"]

In [ ]:
#new column for the total combined variant number from CIViC and MOA
merged_all_evidence_df["Sum of total Evidence from CIViC and MOA"] = merged_evidence_total

In [ ]:
#new percentage of each category of the total merged variants from CIViC and MOA
merged_all_evidence_df["Merged Evidence Percentage"]= (merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"] / merged_all_evidence_df["Sum of total Evidence from CIViC and MOA"])*100
merged_all_evidence_df=merged_all_evidence_df.round({"Merged Evidence Percentage":2})
merged_all_evidence_df["Merged Evidence Percentage"]= merged_all_evidence_df["Merged Evidence Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_all_evidence_df["Percentage of all Merged Evidence Items"]= merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"].astype(str)+" / "+merged_all_evidence_df["Sum of total Evidence from CIViC and MOA"].astype(str)+" ("+merged_all_evidence_df["Merged Evidence Percentage"].astype(str)+")"

Summary Table 8: The table below shows what percentage of all evidence items in merged CIViC and MOA data are associated with Normalized, Unable to Normalize, and Not Supported variants. This will not add up to 100% because evidence itmes may be used across multiple variants.

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_8 = merged_all_evidence_df.drop(["Count of CIViC Evidence Items per Category", "Percentage of all CIViC Evidence Items", "Count of MOA Assertions per Category", "Percentage of all MOA Assertions", "Sum of Evidence from CIViC and MOA per Category", "Sum of total Evidence from CIViC and MOA","Merged Evidence Percentage" ], axis=1)
merged_civic_moa_summary_table_8= merged_civic_moa_summary_table_8.set_index("Variant Category")
merged_civic_moa_summary_table_8

In [ ]:
#new percentage of CIViC contribution for each category
merged_all_evidence_df["CIViC Evidence of Category Percentage"]=(merged_all_evidence_df["Count of CIViC Evidence Items per Category"] / merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"])*100
merged_all_evidence_df=merged_all_evidence_df.round({"CIViC Evidence of Category Percentage":2})
merged_all_evidence_df["CIViC Evidence of Category Percentage"]= merged_all_evidence_df["CIViC Evidence of Category Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_all_evidence_df["Percentage of CIViC Evidence of Category"]=merged_all_evidence_df["Count of CIViC Evidence Items per Category"].astype(str)+" / "+merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"].astype(str)+" ("+merged_all_evidence_df["CIViC Evidence of Category Percentage"].astype(str)+")"

In [ ]:
#new percentage of MOA contribution for each category
merged_all_evidence_df["MOA Evidence of Category Percentage"]=(merged_all_evidence_df["Count of MOA Assertions per Category"] / merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"])*100
merged_all_evidence_df=merged_all_evidence_df.round({"MOA Evidence of Category Percentage":2})
merged_all_evidence_df["MOA Evidence of Category Percentage"]= merged_all_evidence_df["MOA Evidence of Category Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_all_evidence_df["Percentage of MOA Evidence of Category"]=merged_all_evidence_df["Count of MOA Assertions per Category"].astype(str)+" / "+merged_all_evidence_df["Sum of Evidence from CIViC and MOA per Category"].astype(str)+" ("+merged_all_evidence_df["MOA Evidence of Category Percentage"].astype(str)+")"

Summary Table 9: The table below shows what percentage of the merged data originates from CIViC and MOA sources

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_9 = merged_all_evidence_df.drop(["Count of CIViC Evidence Items per Category", "Percentage of all CIViC Evidence Items", "Count of MOA Assertions per Category", "Percentage of all MOA Assertions", "Sum of Evidence from CIViC and MOA per Category", "Sum of total Evidence from CIViC and MOA","Merged Evidence Percentage","Percentage of all Merged Evidence Items","CIViC Evidence of Category Percentage", "MOA Evidence of Category Percentage" ], axis=1)
merged_civic_moa_summary_table_9= merged_civic_moa_summary_table_9.set_index("Variant Category")
merged_civic_moa_summary_table_9

In [ ]:
#import summary tables from source notebooks
for_merge_not_supported_variant_evidence_percent_of_civic_df = pd.read_csv("../analysis/for_merge_not_supported_variant_evidence_percent_of_civic_df.csv", sep=",")
for_merge_not_supported_feature_assertion_df = pd.read_csv("../analysis/for_merge_not_supported_feature_assertion_df.csv", sep=",")

In [ ]:
#merge CIViC and MOA summary tables
merged_not_supported_evidence_df = pd.merge(for_merge_not_supported_variant_evidence_percent_of_civic_df, for_merge_not_supported_feature_assertion_df, on="Category",how="outer")
merged_not_supported_evidence_df = merged_not_supported_evidence_df.replace(np.nan, 0, regex=True)

Summary Table 10: The table below shows the categories of Not Supported variants and what percentage of source (CIViC or MOA) evidence items are associated with those variants. These percentages will not add up to 100% because Not Supported variants make up 44.11% of CIViC variants and 63.09% of MOA variants and evidence itmes may be used across multiple variants.(Reference Summary Table 7- merged_civic_moa_summary_table_7)

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_10= merged_not_supported_evidence_df.drop(["Count of CIViC Evidence Items per Category","Count of MOA Assertions per Category"], axis=1)
merged_civic_moa_summary_table_10= merged_civic_moa_summary_table_10.set_index("Category")
merged_civic_moa_summary_table_10

In [ ]:
#add up evidence from CIViC and MOA for each Variant Category
merged_not_supported_evidence_df["Sum of Evidence from CIViC and MOA per Category"]=merged_not_supported_evidence_df["Count of CIViC Evidence Items per Category"]+merged_not_supported_evidence_df["Count of MOA Assertions per Category"]
#new column for the total combined evidence number from CIViC and MOA
merged_not_supported_evidence_df["Sum of total Evidence from CIViC and MOA"]= int(merged_evidence_total)

In [ ]:
#new percentage of each category of the total merged evidence from CIViC and MOA
merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Merged Total Percentage"]= (merged_not_supported_evidence_df["Sum of Evidence from CIViC and MOA per Category"] / merged_not_supported_evidence_df["Sum of total Evidence from CIViC and MOA"])*100
merged_not_supported_evidence_df=merged_not_supported_evidence_df.round({"Merged Not Supported Evidence Category of Merged Total Percentage":2})
merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Merged Total Percentage"]= merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Merged Total Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_not_supported_evidence_df["Percentage of all Merged Evidence Items"]= merged_not_supported_evidence_df["Sum of Evidence from CIViC and MOA per Category"].astype(str)+" / "+merged_not_supported_evidence_df["Sum of total Evidence from CIViC and MOA"].astype(str)+" ("+merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Merged Total Percentage"].astype(str)+")"

Summary Table 11: The table below shows the evidence items associated with categories of Not Supported variants and what percentage of all evidence items (CIViC and MOA) they make up. These percentages will not add up to 100% because evidence items associated with Not Supported variants make up 45.69% of all variants (CIViC and MOA). 

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_11 = merged_not_supported_evidence_df.drop(["Count of CIViC Evidence Items per Category", "Percent of all CIViC Evidence Items", "Count of MOA Assertions per Category", "Percent of all MOA Assertions", "Sum of Evidence from CIViC and MOA per Category", "Sum of total Evidence from CIViC and MOA", "Merged Not Supported Evidence Category of Merged Total Percentage"], axis=1)
merged_civic_moa_summary_table_11=merged_civic_moa_summary_table_11.set_index("Category")
merged_civic_moa_summary_table_11

In [ ]:
#add up all evidence, for total evidence items associated with Not Supported Variants
merged_not_supported_evidence_total= merged_not_supported_evidence_df["Sum of Evidence from CIViC and MOA per Category"].sum()
merged_not_supported_evidence_df["Merged Not Supported Evidence Total"]=merged_not_supported_evidence_total

In [ ]:
#new percentage of each category of the total merged evidence from CIViC and MOA
merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Not Supported Percentage"]= (merged_not_supported_evidence_df["Sum of Evidence from CIViC and MOA per Category"] / merged_not_supported_evidence_df["Merged Not Supported Evidence Total"])*100
merged_not_supported_evidence_df=merged_not_supported_evidence_df.round({"Merged Not Supported Evidence Category of Not Supported Percentage":2})
merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Not Supported Percentage"]= merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Not Supported Percentage"].astype(str) + "%"

In [ ]:
#merge fraction and percentage
merged_not_supported_evidence_df["Percentage of all Not Supported Evidence Items"]= merged_not_supported_evidence_df["Sum of Evidence from CIViC and MOA per Category"].astype(str)+" / "+merged_not_supported_evidence_df["Merged Not Supported Evidence Total"].astype(str)+" ("+merged_not_supported_evidence_df["Merged Not Supported Evidence Category of Not Supported Percentage"].astype(str)+")"

Summary Table 12: The table below shows the categories of Not Supported variants the percent of each category of total merged Not Supported evidence.

In [ ]:
#clean up summary table by dropping columns and resetting index
merged_civic_moa_summary_table_12 = merged_not_supported_evidence_df.drop(["Count of CIViC Evidence Items per Category", "Percent of all CIViC Evidence Items", "Count of MOA Assertions per Category", "Percent of all MOA Assertions", "Sum of Evidence from CIViC and MOA per Category", "Sum of total Evidence from CIViC and MOA","Percentage of all Merged Evidence Items","Merged Not Supported Evidence Category of Merged Total Percentage", "Merged Not Supported Evidence Total","Merged Not Supported Evidence Category of Not Supported Percentage"], axis=1)
merged_civic_moa_summary_table_12=merged_civic_moa_summary_table_12.set_index("Category")
merged_civic_moa_summary_table_12

## Merge CIViC and MOA Summary Impact Dataframes

In [ ]:
not_supported_variant_impact_df = pd.read_csv("../analysis/not_supported_variant_impact_df.csv", sep=",")
not_supported_feature_impact_df = pd.read_csv("../analysis/not_supported_feature_impact_df.csv", sep=",")

In [ ]:
merged_not_supported_impact_df = pd.merge(not_supported_feature_impact_df, not_supported_variant_impact_df, on='Category')

### Merge aspects of the dataframe (number of evidence items, variants, impact score)

In [ ]:
merged_not_supported_impact_df['Sum Evidence Items'] = merged_not_supported_impact_df['Total Number Evidence Items']+merged_not_supported_impact_df['Total Number Assertions']

In [ ]:
merged_not_supported_impact_df['Sum Variants'] = merged_not_supported_impact_df['Total Number Variants']+merged_not_supported_impact_df['Total Number Features']

In [ ]:
merged_not_supported_impact_df['Sum Impact Score'] = merged_not_supported_impact_df['CIVIC Total Sum Impact Score']+merged_not_supported_impact_df['MOA Total Sum Impact Score']
merged_not_supported_impact_df

### Calculate the ratio of features/variants that come from MOA

In [ ]:
merged_not_supported_impact_df['Ratio of MOA Features to MOA+CIVIC per Category'] = round(merged_not_supported_impact_df['Total Number Features']/(merged_not_supported_impact_df['Total Number Features']+merged_not_supported_impact_df['Total Number Variants']), 2)

In [ ]:
merged_not_supported_impact_df

In [ ]:
merged_not_supported_impact_df.to_csv('../merged_not_supported_impact_df.csv', index=False)

In [ ]:
trimmed_merged_not_supported_impact_df = merged_not_supported_impact_df.drop(['MOA Total Sum Impact Score', 'Average Impact Score per Feature', 'Average Impact Score per Assertion', 'Total Number Assertions', 'Total Number Features', 'CIVIC Total Sum Impact Score', 'Average Impact Score per Variant', 'Average Impact Score per Evidence Item', 'Total Number Evidence Items', '% Accepted Evidence Items', 'Total Number Variants'], axis=1)
trimmed_merged_not_supported_impact_df

In [ ]:
trimmed_merged_not_supported_impact_df.to_csv('../trimmed_merged_not_supported_impact_df.csv', index=False)

In [ ]:
ratio_list = merged_not_supported_impact_df["Ratio of MOA Features to MOA+CIVIC per Category"].to_numpy()

In [ ]:
moa_civic_evidence_sum_list = merged_not_supported_impact_df["Sum Evidence Items"].to_numpy()

In [ ]:
moa_civic_variant_sum_list = merged_not_supported_impact_df["Sum Variants"].to_numpy()

In [ ]:
moa_civic_impact_score_list = merged_not_supported_impact_df["Sum Impact Score"].to_numpy()

In [ ]:
dict_summary = merged_not_supported_impact_df.to_dict('list')
dict_summary

In [ ]:
# cat_to_coords = dict()
# for v in dict_summary["Category"]:
#     cat_to_coords[v] = None
# cat_to_coords

In [ ]:
cat_to_coords = dict()
for i, row in merged_not_supported_impact_df.iterrows():
    cat_to_coords[row[0]] = f"{(row[12], row[14])}"
cat_to_coords


In [ ]:

# cat_to_coords = {v: None for v in dict["Category"]}

# for k, v in cat_to_coords.items():  
    
#     for v in dict["Sum Evidence Items"]:
        
# cat_to_coords

# INSERT REFACTOR WORK

In [ ]:
fig5, ax = plt.subplots(figsize=(25, 15))

colors = [
    "red", "darkorange", "forestgreen", "lawngreen", "gold", "cyan",
     "deepskyblue", "mediumslateblue", "blue", "pink", "deeppink", "purple"
]

for i in range(12):
    ratio = ratio_list[i]
    size = moa_civic_variant_sum_list[i] * 20

    # Both will always have b
    x = [0] + np.cos(np.linspace(2 * np.pi * ratio, 2 * np.pi, 100)).tolist()
    y = [0] + np.sin(np.linspace(2 * np.pi * ratio, 2 * np.pi, 100)).tolist()
    marker_b = np.column_stack([x, y])

    if ratio:
        # use both a & b
        x = [0] + np.cos(np.linspace(0, 2 * np.pi * ratio, 100)).tolist()
        y = [0] + np.sin(np.linspace(0, 2 * np.pi * ratio, 100)).tolist()
        marker_a = np.column_stack([x, y])

        ax.scatter(
            moa_civic_evidence_sum_list[i], moa_civic_impact_score_list[i], marker=marker_a, s=size,
            facecolor=colors[i], edgecolors='black', hatch='///////'
        )

    ax.scatter(
        moa_civic_evidence_sum_list[i],moa_civic_impact_score_list[i], marker=marker_b, s=size, facecolor=colors[i]
    )

plt.show()


# END REFACTOR WORK

In [ ]:
# first define the ratios
r1 = ratio_list[0]  
r2=ratio_list[1]
r3=ratio_list[2] 
r4=ratio_list[3]  
r5=ratio_list[4]
r6=ratio_list[5]
r7=ratio_list[6]
r8=ratio_list[7]
r9=ratio_list[8]
r10=ratio_list[9] 
r11=ratio_list[10] 
r12=ratio_list[11]

# define some sizes of the scatter marker
size1=moa_civic_variant_sum_list[0]*20  
size2=moa_civic_variant_sum_list[1]*20 
size3=moa_civic_variant_sum_list[2]*20
size4=moa_civic_variant_sum_list[3]*20 
size5=moa_civic_variant_sum_list[4]*20 
size6=moa_civic_variant_sum_list[5]*20 
size7=moa_civic_variant_sum_list[6]*20 
size8=moa_civic_variant_sum_list[7]*20 
size9=moa_civic_variant_sum_list[8]*20 
size10=moa_civic_variant_sum_list[9]*20  
size11=moa_civic_variant_sum_list[10]*20  
size12=moa_civic_variant_sum_list[11]*20 

# calculate the points of the pie markers
#
# these are the origin (0,0) +
# some points on a circle cos,sin

## if no MOA features in category, only one segement

x = [0] + np.cos(np.linspace(0, 2 * np.pi * r1, 100)).tolist()
y = [0] + np.sin(np.linspace(0, 2 * np.pi * r1, 100)).tolist()
xy1a = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r1, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r1, 2 * np.pi, 100)).tolist()
xy1b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r2, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r2, 2 * np.pi, 100)).tolist()
xy2b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r3, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r3, 2 * np.pi, 100)).tolist()
xy3b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(0, 2 * np.pi * r4, 100)).tolist()
y = [0] + np.sin(np.linspace(0, 2 * np.pi * r4, 100)).tolist()
xy4a = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r4, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r4, 2 * np.pi, 100)).tolist()
xy4b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r5, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r5, 2 * np.pi, 100)).tolist()
xy5b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(0, 2 * np.pi * r6, 100)).tolist()
y = [0] + np.sin(np.linspace(0, 2 * np.pi * r6, 100)).tolist()
xy6a = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r6, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r6, 2 * np.pi, 100)).tolist()
xy6b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(0, 2 * np.pi * r7, 100)).tolist()
y = [0] + np.sin(np.linspace(0, 2 * np.pi * r7, 100)).tolist()
xy7a = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r7, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r7, 2 * np.pi, 100)).tolist()
xy7b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(0, 2 * np.pi * r8, 100)).tolist()
y = [0] + np.sin(np.linspace(0, 2 * np.pi * r8, 100)).tolist()
xy8a = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r8, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r8, 2 * np.pi, 100)).tolist()
xy8b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r9, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r9, 2 * np.pi, 100)).tolist()
xy9b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r10, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r10, 2 * np.pi, 100)).tolist()
xy10b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r11, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r11, 2 * np.pi, 100)).tolist()
xy11b = np.column_stack([x, y])

x = [0] + np.cos(np.linspace(2 * np.pi * r12, 2 * np.pi, 100)).tolist()
y = [0] + np.sin(np.linspace(2 * np.pi * r12, 2 * np.pi, 100)).tolist()
xy12b = np.column_stack([x, y])


fig5, ax = plt.subplots(figsize=(25, 15))

ax.scatter(moa_civic_evidence_sum_list[0],moa_civic_impact_score_list[0], marker=xy1a,
           s=size1, facecolor='red', edgecolors='black', hatch='///////' )
ax.scatter(moa_civic_evidence_sum_list[0],moa_civic_impact_score_list[0], marker=xy1b,
           s=size1, facecolor='red')


ax.scatter(moa_civic_evidence_sum_list[1],moa_civic_impact_score_list[1], marker=xy2b,
           s=size2, facecolor='darkorange')


ax.scatter(moa_civic_evidence_sum_list[2],moa_civic_impact_score_list[2], marker=xy3b,
           s=size3, facecolor='forestgreen')

ax.scatter(moa_civic_evidence_sum_list[3],moa_civic_impact_score_list[3], marker=xy4a,
           s=size4, facecolor='lawngreen', edgecolors='black', hatch='///////')
ax.scatter(moa_civic_evidence_sum_list[3],moa_civic_impact_score_list[3], marker=xy4b,
           s=size4, facecolor='lawngreen')


ax.scatter(moa_civic_evidence_sum_list[4],moa_civic_impact_score_list[4], marker=xy5b,
           s=size5, facecolor='gold')

ax.scatter(moa_civic_evidence_sum_list[5],moa_civic_impact_score_list[5], marker=xy6a,
           s=size6, facecolor='cyan', edgecolors='black', hatch='///////')
ax.scatter(moa_civic_evidence_sum_list[5],moa_civic_impact_score_list[5], marker=xy6b,
           s=size6, facecolor='cyan')

ax.scatter(moa_civic_evidence_sum_list[6],moa_civic_impact_score_list[6], marker=xy7a,
           s=size7, facecolor='deepskyblue' , edgecolors='black', hatch='///////')
ax.scatter(moa_civic_evidence_sum_list[6],moa_civic_impact_score_list[6], marker=xy7b,
           s=size7, facecolor='deepskyblue')

ax.scatter(moa_civic_evidence_sum_list[7],moa_civic_impact_score_list[7], marker=xy8a,
           s=size8, facecolor='mediumslateblue', edgecolors='black', hatch='///////' )
ax.scatter(moa_civic_evidence_sum_list[7],moa_civic_impact_score_list[7], marker=xy8b,
           s=size8, facecolor='mediumslateblue')


ax.scatter(moa_civic_evidence_sum_list[8],moa_civic_impact_score_list[8], marker=xy9b,
           s=size9, facecolor='blue')


ax.scatter(moa_civic_evidence_sum_list[9],moa_civic_impact_score_list[9], marker=xy10b,
           s=size10, facecolor='pink')


ax.scatter(moa_civic_evidence_sum_list[10],moa_civic_impact_score_list[10], marker=xy11b,
           s=size11, facecolor='deeppink')


ax.scatter(moa_civic_evidence_sum_list[11],moa_civic_impact_score_list[11], marker=xy12b,
           s=size12, facecolor='purple')


legend_elements = [Line2D([0], [0], marker='o', color='w', label='Expression (626, 3629.00)',
                          markerfacecolor='red', markersize=20),
                   Line2D([0], [0], marker='o', color='w', label='Epigenetic Modification (24, 285.50)',
                          markerfacecolor='darkorange', markersize=20),
                   Line2D([0], [0], marker='o', color='w', label='Fusion (1239, 5689.25)',
                          markerfacecolor='forestgreen', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Protein Consequence (705, 3747.75)',
                          markerfacecolor='lawngreen', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Gene Function (347, 1822.50)',
                          markerfacecolor='gold', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Rearrangements (218, 945.00)',
                          markerfacecolor='cyan', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Copy Number (93, 254.00)',
                          markerfacecolor='deepskyblue', markersize=20),
                   Line2D([0], [0], marker='o', color='w', label='Other  (185, 708.50)',
                          markerfacecolor='mediumslateblue', markersize=20),
                   Line2D([0], [0], marker='o', color='w', label='Genotypes Easy (23, 195.00)',
                          markerfacecolor='blue', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Genotypes Compound (7, 117.50)',
                          markerfacecolor='pink', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Region Defined (924, 8311.50)',
                          markerfacecolor='deeppink', markersize=20),       
                   Line2D([0], [0], marker='o', color='w', label='Transcript (471, 346.50)',
                          markerfacecolor='purple', markersize=20),
                   Line2D([0], [0], color='w',
                          markerfacecolor='white'),       
                   mpatches.Circle(xy=(0,0), radius=1, edgecolor="black", facecolor= "white", hatch='///////', label='Variants from MOA'), 
                   mpatches.Circle(xy=(0,0), radius=1, edgecolor="black", facecolor= "white", label='Variants from CIViC')]
legend_2_elements= [Line2D([0], [0], color='w',
                          markerfacecolor='white'),
                     Line2D([0], [0], color='w',
                          markerfacecolor='white'),
                     Line2D([0], [0], color='w',
                          markerfacecolor='white'),
                     Line2D([0], [0], marker='o', color='w', label='          300 variants per category',
                          markerfacecolor='black', markersize=85),
                     Line2D([0], [0], color='k',
                          markerfacecolor='white'),
                     Line2D([0], [0], color='w',
                          markerfacecolor='white'),
                     Line2D([0], [0], color='w',
                          markerfacecolor='white'),
                     Line2D([0], [0], marker='o', color='w', label='          200 variants per category',
                          markerfacecolor='black', markersize=60),
                     Line2D([0], [0], color='k',
                          markerfacecolor='white'),
                     Line2D([0], [0], color='w',
                          markerfacecolor='white'),
                     
                     Line2D([0], [0], marker='o', color='w', label='          50 variants per category',
                          markerfacecolor='black', markersize=24),
                     Line2D([0], [0], color='w',
                          markerfacecolor='white')]

# Create the figure
first_legend = ax.legend(handles=legend_elements,loc='lower left', bbox_to_anchor=(0., .325, 3, .550), fontsize= 20)
# first_legend._legend_box.align = "left"
ax.add_artist(first_legend)
ax.set_xlabel('Number of Evidence Items per Category (MOA & CIViC)', fontsize= 25)
ax.set_ylabel('Impact Score per Category (MOA & CIViC)', fontsize= 25)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(4)
ax.tick_params(width=4)
second_legend = ax.legend(handles=legend_2_elements, loc='lower right', frameon=False)
ax.add_artist(second_legend)
plt.title("Impact Score of Currently Not Supported Variant Categories", fontsize=40)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.setp(plt.gca().get_legend().get_texts(), fontsize='17')

plt.savefig('myfig2.jpeg', dpi=1000)



plt.show()